In [1]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [2]:
from astropy.io import fits

In [3]:
#import astroprov
import numpy as np
import os

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine
import sys


In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m'),("spark.sql.execution.arrow.enabled", "true")]) # ("spark.sql.execution.arrow.enabled", "true")


In [5]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)

In [6]:

mags = [7,6,5,4,3,2,1]
for mag in mags:
    dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist_{}".format(mag)).option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
    dataframe_mysql.registerTempTable('starlist_{}'.format(mag))

In [18]:
def innerJoin(tableName,provDir,totGross,variableList,diffSize=1,xlength=1015.,ylength=1085.):
    nsegs = 4
    mag = variableList[6]
    temptableName = tableName[0:5]+tableName[-1]
    df = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE, ELLIPTICITY FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(temptableName,abs(mag)))
    df.registerTempTable(temptableName+'match')
    sqlContext.cacheTable(temptableName+'match')
    totNumber = df.count()
    #print('tot',totNumber)
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    
    #totSeg = []
    #df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    for xsegs in range(nsegs):
        xmin = int(xsegs*xsegment)+15
        xmax = int(xmin+xsegment)+15
        for ysegs in range(nsegs):
            ymin = int(ysegs*ysegment)+15
            ymax = int(ymin+ysegment)+15
            
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName+'match',xmin,xmax,ymin,ymax))
            number = df.count()
            variableList.append(number)
            
#             dftot = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName,xmin,xmax,ymin,ymax))
#             number = df.count()
#             #variableList.append(number)
#             totSeg.append(number)

            # need to create some place to write to

    
    
#    variableList.extend(totSeg)
    #astroprov.provcall([tableName,'starlist_{}'.format(abs(mag))],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)
    
    sqlContext.uncacheTable(temptableName+'match')
    sqlContext.dropTempTable(temptableName+'match')
    sqlContext.uncacheTable(temptableName)
    sqlContext.dropTempTable(temptableName)
    variableList.append(totNumber)
    variableList.append(totGross)
    variableTuple = tuple(variableList)
    #print(variableTuple)
    return variableTuple

def makeConfig(valList,provDir,tableName='table',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    

    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test1.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_1.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    #astroprov.provcall(valList,[confName],"makeConfig_Python2Python_SQ_tmpl.provn","makeConfig",provDir)
    return(confName,tableName,catname)



def findObjects(confName,tableName,catName,provDir,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',imagename='/home/mj1e16/iraf/editedImage5000.fits'):
    
    #astroprov.provcall([confName,'/home/mj1e16/iraf/editedImage5000.fits'],[tableName],"findObjects_Python2DaoStarfidner_SQ_tmpl.provn","findObjects",provDir)

    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    df_spark = sqlContext.createDataFrame(df2)
    temptableName = tableName[0:5]+tableName[-1]
    #print(temptableName)
    df_spark.registerTempTable(temptableName)
    totGross = df_spark.count()
    #print('Gross ',totGross)
    sqlContext.cacheTable(temptableName)
    # count rows in table and add to some big table
    
    #df_spark.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    return totGross

def alltogethernow(valList,tableName,IMAGE,provDir,variableList):
    Names = makeConfig(valList,provDir,tableName=tableName)
    totGross = findObjects(Names[0],Names[1],Names[2],provDir,imagename=IMAGE) # confName tabName catname
    finalTuple = innerJoin(Names[1],provDir,totGross,variableList)
    return finalTuple

In [8]:
def hotpantsQuality(image):
    hdu = fits.open(image)
    imData = hdu[0].data
    imData = abs(imData)
    totVal = sum(imData)
    totVal = sum(totVal)
    return totVal

In [9]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]]

In [10]:
def evaluateImage(valList,simImage,provDir,ccd,median,norm,minmag,smallName='Poshak',atList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    
    finalTableTuples = []
    t0 = time.time()

    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                for x2 in range(len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    for x3 in range(len(valList[3])):
                        variableList = [float(valList[0][x0]),float(valList[1][x1]),float(valList[2][x2]),float(valList[3][x3]),ccd,median,minmag,norm]
                        fullname = name+str(x3)
                        fullValList = [valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]]
                        print(fullname)
                        try:
                            finalTableTuples.append(alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,provDir=provDir,variableList=variableList))
                        except RuntimeError:
                            with open('redos','a') as f:
                                bigString = simImage+','+str(fullname)+','+str(fullValList)
                                f.write(bigString)
                            print('Redo with valList = ',fullValList)
                            pass

    print(finalTableTuples)
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],backSize=x[2],backFiltersize=x[3],ccd=x[4],median=x[5],medianQuality=x[6],
                                   minmag=x[7], xy_0=int(x[8]),xy_1=int(x[9]),xy_2=int(x[10]),xy_3=int(x[11]),xy_4=int(x[12]),
                                   xy_5=int(x[13]),xy_6=int(x[14]),xy_7=int(x[15]),xy_8=int(x[16]),xy_9=int(x[17]),xy_10=int(x[18]),
                                   xy_11=int(x[19]),xy_12=int(x[20]),xy_13=int(x[21]),xy_14=int(x[22]),xy_15=int(x[23]),totNum=int(x[24]),totGross=int(x[25])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResults_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    print(time.time()-t0)


In [11]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[1,3,5,7,9,11]]

In [12]:
#sqlContext.cacheTable('starlist')

In [13]:
#sqlContext.sql('DROP TABLE IF EXISTS finalResults_44_1')
#valList = [[1.],[1.],[8.],[3.]]
#valList = [np.linspace(1,2,2),np.linspace(1,2,2),[8.],[3.]]

In [14]:
# ccd = [44,63,79]
# median = [0,1,2]
# mags = np.linspace(-7,-1,7)

#ccd = [44] #,63,79]
ccd = [79]
median = [2]
mags = [-7,-6,-5,-4,-3,-2,-1]


In [15]:

norms = []
for inc,c in enumerate(ccd):
    totVals = []
    for x in range(3):
        totVals.append(hotpantsQuality('/home/mj1e16/Simages/diff_{}_{}.fits'.format(c,x)))
    norms.append([float(y/totVals[1]) for y in totVals])
print(norms)

[[3.944419917268288, 1.0, 1.392786645096585]]


In [16]:
norms[0][2]

1.392786645096585

In [19]:
### Make function to replace starlist
for inc,c in enumerate(ccd):
    for inmed,med in enumerate(median):
        norm = norms[0][2]
        subprocess.call(['mkdir','/home/mj1e16/keplerPhotometry/provDump/{}/{}'.format(c,med)])
        for mag in mags:
            sqlContext.cacheTable('starlist_{}'.format(abs(mag)))
            subprocess.call(['mkdir','/data/mj1e16/provDump/{}/{}/{}'.format(c,med,mag)])
            imName = '/home/mj1e16/Simages/diff_{}_{}{}_alt.fits'.format(c,med,mag)
            evaluateImage(valList,imName,'/data/mj1e16/provDump/{}/{}/{}/'.format(c,med,mag),'ccd_{}'.format(c),'median_{}'.format(med),norm,mag,smallName='Poshak_{}_{}_{}'.format(c,med,mag))
            sqlContext.uncacheTable('starlist_{}'.format(abs(mag)))

Poshak_79_2_-7_0_0_0_0


ParseException: u"\nmismatched input 'INNER' expecting <EOF>(line 1, pos 55)\n\n== SQL ==\nSELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE, FROM Posha0 INNER JOIN starlist_7 ON Posha0.X_IMAGE BETWEEN starlist_7.X_POS_MIN AND starlist_7.X_POS_MAX AND Posha0.Y_IMAGE BETWEEN starlist_7.Y_POS_MIN AND starlist_7.Y_POS_MAX\n-------------------------------------------------------^^^\n"

In [ ]:
# change prov directory back on to ssd

In [49]:
a = [1.0, 1.0, 8.0, 3.0, 'ccd_44', 'median_1', -7, 1.0, 67, 62, 64, 48, 70, 67, 68, 51, 70, 67, 64, 30, 70, 67, 68, 48, 67, 62, 64, 48, 70, 67, 68, 51, 70, 67, 64, 30, 70, 67, 68, 48, 967, 15760]

In [32]:
a = [1.0, 1.0, 8.0, 3.0, 'ccd_44', 'median_1', -7, 1.0, 67, 67, 62, 62, 64, 64, 48, 48, 70, 70, 67, 67, 68, 68, 51, 51, 70, 70, 67, 67, 64, 64, 30, 30, 70, 70, 67, 67, 68, 68, 48, 48, 967, 15760]

In [33]:
a[24]

70

In [14]:
tup = [(1,2,3),(4,5,6)]
rdd = sc.parallelize(tup)
kepler = rdd.map(lambda x: Row(one=int(x[0]),two=int(x[1]),three=int(x[2])))
schemaKepler = sqlContext.createDataFrame(kepler)
schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='testTab',user='mj1e16',password='[sqlT1G3R]').mode('append').save()


In [29]:
a = sqlContext.sql("show tables").show()
print(a)

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |  starlist|       true|
|        |starlist_1|       true|
|        |starlist_2|       true|
|        |starlist_3|       true|
|        |starlist_4|       true|
|        |starlist_5|       true|
|        |starlist_6|       true|
|        |starlist_7|       true|
+--------+----------+-----------+

None


In [15]:
def test():
    row = [('tabName',1,1),('tabName2',1,2)]
    rdd = sc.parallelize(row)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xyRegion=int(x[1]),magRegion=int(x[2])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    df = schemaKepler.toPandas()
    print(df)

In [25]:
def testPartial(value):
    return (value) #,value)

In [26]:
p = ThreadPool(6)
values = [0,1,2,3,4,5]
answer = []
for x in range(5):
    answer.extend(p.map(testPartial,values))
print(answer)

[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]


In [26]:
import sys

In [27]:
sys.getsizeof(answer)

336

In [12]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
    maxVal = df2['MAG_BEST'].max()
    minVal = df2['MAG_BEST'].min()
#     df4 = df2.sort_values('MAG_BEST')
#     return df4
    print(minVal,maxVal)

In [32]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
#     maxVal = df2['MAG_BEST'].max()
#     minVal = df2['MAG_BEST'].min()
    df4 = df2.sort_values('MAG_BEST')
    return df4
#     print(minVal,maxVal)

In [15]:
for x in range(6):
    print('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))
    findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

/home/mj1e16/sextractor/sextractor-master/config/test0.cat
/home/mj1e16/sextractor/sextractor-master/config/test1.cat
/home/mj1e16/sextractor/sextractor-master/config/test2.cat
/home/mj1e16/sextractor/sextractor-master/config/test3.cat
/home/mj1e16/sextractor/sextractor-master/config/test4.cat
/home/mj1e16/sextractor/sextractor-master/config/test5.cat


In [33]:
x = 2
df = findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

In [35]:
df

,NUMBER,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY
3025,3026,13.0256,782.0090,185.0009,0.352
2869,2870,13.0305,1052.8906,184.9989,0.036
13573,13574,13.0327,1052.7808,946.0535,0.125
13344,13345,13.0328,239.7816,946.0737,0.031
6969,6970,13.0329,239.5148,437.8912,0.103
11193,11194,13.0329,239.6400,691.9606,0.054
13666,13667,13.0333,781.5560,946.0166,0.260
6968,6969,13.0335,781.4474,438.1546,0.157
7010,7011,13.0338,510.7693,437.9970,0.031
2859,2860,13.0340,239.8327,185.1199,0.095


In [49]:
magRange = np.linspace(15,30,16)

In [50]:
magRange

array([ 15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,
        26.,  27.,  28.,  29.,  30.])

In [57]:
catName = 'test0.cat'

assoc = Table.read(catName,format='ascii.sextractor')
t0 = time.time()
df = assoc.to_pandas()
#df2 = df[df['MAG_BEST'] != 99.0000]
print(time.time()-t0) #, totGross)


0.00117707252502


In [56]:
t0 = time.time()
df_spark = sqlContext.createDataFrame(df)
print(time.time()-t0) #, totGross)

t0 = time.time()
totGross = df_spark.count()
print(time.time()-t0, totGross)

1.73592019081
(0.06544804573059082, 16094)


In [59]:
!head test0.cat

#   1 NUMBER                 Running object number                                     
#   2 MAG_BEST               Best of MAG_AUTO and MAG_ISOCOR                            [mag]
#   3 X_IMAGE                Object position along x                                    [pixel]
#   4 Y_IMAGE                Object position along y                                    [pixel]
#   5 ELLIPTICITY            1 - B_IMAGE/A_IMAGE                                       
         1  20.7173    744.5268     19.6496    0.438
         2  99.0000   1000.7009     19.5017    0.285
         3  99.0000    133.3164     20.2157    0.325
         4  21.8619    138.0000     20.4564    0.499
         5  21.5303    779.9139     23.0000    0.633


In [27]:
x = 0
vals = [[x],[1],[2]]
val = [[x],[3]]
print(val)
x = 1
val = [[x],[3]]
print(val)
vals.append(val[0])

[[0], [3]]
[[1], [3]]


In [4]:
import numpy as np 
smallName = 'test'

In [5]:
valListminus1 = [1,3,5,7,9,11]
for vals in (valListminus1):
    valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[vals]]

    finalTableTuples = []
    #p = ThreadPool(1)
    #t0 = time.time()
    for x0 in range(3): #len(valList[0])):
        for x1 in range(3): #len(valList[1])):
                for x2 in range(3): #len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    print(fullValList)
#                     try:
#                         finalTableTuples.extend(p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList))

test_0_0_0_0
[[1.0, 1.0, 8.0, 1]]
test_0_0_1_0
[[1.0, 1.0, 16.0, 1]]
test_0_0_2_0
[[1.0, 1.0, 24.0, 1]]
test_0_1_0_0
[[1.0, 2.0, 8.0, 1]]
test_0_1_1_0
[[1.0, 2.0, 16.0, 1]]
test_0_1_2_0
[[1.0, 2.0, 24.0, 1]]
test_0_2_0_0
[[1.0, 3.0, 8.0, 1]]
test_0_2_1_0
[[1.0, 3.0, 16.0, 1]]
test_0_2_2_0
[[1.0, 3.0, 24.0, 1]]
test_1_0_0_0
[[2.0, 1.0, 8.0, 1]]
test_1_0_1_0
[[2.0, 1.0, 16.0, 1]]
test_1_0_2_0
[[2.0, 1.0, 24.0, 1]]
test_1_1_0_0
[[2.0, 2.0, 8.0, 1]]
test_1_1_1_0
[[2.0, 2.0, 16.0, 1]]
test_1_1_2_0
[[2.0, 2.0, 24.0, 1]]
test_1_2_0_0
[[2.0, 3.0, 8.0, 1]]
test_1_2_1_0
[[2.0, 3.0, 16.0, 1]]
test_1_2_2_0
[[2.0, 3.0, 24.0, 1]]
test_2_0_0_0
[[3.0, 1.0, 8.0, 1]]
test_2_0_1_0
[[3.0, 1.0, 16.0, 1]]
test_2_0_2_0
[[3.0, 1.0, 24.0, 1]]
test_2_1_0_0
[[3.0, 2.0, 8.0, 1]]
test_2_1_1_0
[[3.0, 2.0, 16.0, 1]]
test_2_1_2_0
[[3.0, 2.0, 24.0, 1]]
test_2_2_0_0
[[3.0, 3.0, 8.0, 1]]
test_2_2_1_0
[[3.0, 3.0, 16.0, 1]]
test_2_2_2_0
[[3.0, 3.0, 24.0, 1]]
test_0_0_0_0
[[1.0, 1.0, 8.0, 3]]
test_0_0_1_0
[[1.0, 1.0, 16.0,

In [18]:
np.linspace(-7,-1,7)

array([-7., -6., -5., -4., -3., -2., -1.])

In [21]:
a = 1000/(4**2)
print(a)

62


In [22]:
a = [1,7,17,177]
a.index(7)

1

In [23]:
import random


0.08660731433957158

In [25]:
for x in range(19):
    print(random.random())

0.0140653255063
0.802593419661
0.0644352547886
0.687184769617
0.034016480197
0.849046374575
0.106786241334
0.550846086601
0.427172163947
0.939664513971
0.426717565911
0.384613469585
0.763936701066
0.279835598704
0.72149170931
0.979404772458
0.17608765568
0.792341410241
0.93735604297


In [27]:
random.randint(0,20)

13

In [67]:
a = [1,2,3,3,3,3,3,3,3]
# b = a[1]*2
# print(b)
# a = [x*2 for x in a]
# b = a[1]*2
# print(b)

In [68]:
x = 1
b = a
b = [z-1 if a.index(z)==x else z for z in a]
print(a)
print(b)

[1, 2, 3, 3, 3, 3, 3, 3, 3]
[1, 1, 3, 3, 3, 3, 3, 3, 3]


In [46]:
x = 1
while x == 0:
    if x ==1:
        break

In [50]:
delta = -15

while delta < 0:
    for x in range(10):
        print(x)
        while delta < 0:
            delta += 1
            print(delta)
        

0
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9


In [51]:
a = [1,2,1,2,1]
indices = [i for i, x in enumerate(a) if x == 1]

In [52]:
indices

[0, 2, 4]

In [60]:
indices = [0,1,1,5,7]

In [61]:
print(np.median(indices))

1.0


In [73]:
a = [11,11,11,11,5,11,11,11]
b = [-1,1]

for x in range(len(a)):
    for 
        anew = [z for z in a]
        anew[x] += y
#         anewVal = anew[x]+y
#         anew[x] = anewVal
        print(anew)
        #print(a)

[10, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[12, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 10, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 12, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 10, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 12, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 10, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 12, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 4, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 6, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 10, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 12, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 10, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 12, 11]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 10]
[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 12]
[11, 11,

In [64]:
c = [x for x in a]
c[1]+10
print(c)
print(a)

[11, 11, 11, 11, 5, 11, 11, 11]
[11, 11, 11, 11, 5, 11, 11, 11]


In [76]:
a = [1,10,100,-1]
for x in range(len(a)):
    if 0 < a[x] < 99:
        print(a[x])

1
10


In [78]:
a = 10
b = a
a += 10
print(b)
print(a)

10
20


In [79]:
62*16

992